# Install relevant packages

In [1]:
!pip install tables==3.6.1
!pip install socceraction==0.2.0

     |████████████████████████████████| 4.3 MB 4.3 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 241 kB 5.5 MB/s 
  Created wheel for socceraction: filename=socceraction-0.2.0-py3-none-any.whl size=30656 sha256=4f00233f1948a54dba91d26b0eeca2797cff9edba1385f01f6c690de19893415
  Stored in directory: /root/.cache/pip/wheels/ad/b6/3c/8ae71bd96f4cc8ae86a4ee1fdbd7a6e29fc1e59cec98a543a6
Successfully built socceraction


# Store and save data into google drive folder

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%mkdir -p '/content/gdrive/My Drive/Friends of Tracking/'
%cd '/content/gdrive/My Drive/Friends of Tracking/'

Mounted at /content/gdrive
/content/gdrive/My Drive/Friends of Tracking


# Import stuff

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from io import BytesIO
from pathlib import Path
from tqdm.notebook import tqdm
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile

In [5]:
import pandas as pd
from sklearn.metrics import brier_score_loss, roc_auc_score
from xgboost import XGBClassifier

import socceraction.vaep.features as features
import socceraction.vaep.labels as labels

from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value

In [6]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

# Download and preprocess wyscout event data

## Download Data

In [ ]:
data_files={
    'events': 'https://ndownloader.figshare.com/files/14464685',  # ZIP file containing one JSON file for each competition
    'matches': 'https://ndownloader.figshare.com/files/14464622',  # ZIP file containing one JSON file for each competition
    'players': 'https://ndownloader.figshare.com/files/15073721',  # JSON file
    'teams': 'https://ndownloader.figshare.com/files/15073697'  # JSON file
}

In [ ]:
for url in tqdm(data_files.values()):
    url_s3 = urlopen(url).geturl()
    path = Path(urlparse(url_s3).path)
    file_name = path.name
    file_local, _ = urlretrieve(url_s3, file_name)
    if is_zipfile(file_local):
        with ZipFile(file_local) as zip_file:
            zip_file.extractall()

## Preprocess the data

In [7]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

### Teams

In [8]:
json_teams = read_json_file('teams.json')
df_teams = pd.read_json(json_teams)

In [9]:
df_teams.head(10)

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alavés,696,Deportivo Alavés,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
5,Troyes,Troyes,3795,Espérance Sportive Troyes Aube Champagne,"{'name': 'France', 'id': '250', 'alpha3code': ...",club
6,Getafe (Madrid),Getafe,698,Getafe Club de Fútbol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
7,Mönchengladbach,Borussia M'gladbach,2454,Borussia VfL Mönchengladbach,"{'name': 'Germany', 'id': '276', 'alpha3code':...",club
8,"Huddersfield, West Yorkshire",Huddersfield Town,1673,Huddersfield Town FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
9,Bilbao,Athletic Club,678,Athletic Club Bilbao,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [10]:
df_teams.to_hdf('wyscout.h5', key = 'teams', mode = 'w')

### Players

In [11]:
json_players = read_json_file('players.json')
df_players = pd.read_json(json_players)

In [12]:
df_players.head(10)

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konaté,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konaté,null
5,"{'name': 'Netherlands', 'id': '528', 'alpha3co...",83,Jasper,,Cillessen,676,1989-04-22,185,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Netherlands', 'id': '528', 'alpha3co...",33,right,J. Cillessen,664
6,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",91,Toby,,Alderweireld,1624,1989-03-02,187,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",36,right,T. Alderweireld,5629
7,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",88,Jan,,Vertonghen,1624,1987-04-24,189,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",48,left,J. Vertonghen,5629
8,"{'name': 'France', 'id': '250', 'alpha3code': ...",74,Alexander,,Djiku,3783,1994-08-09,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",229427,right,A. Djiku,null
9,"{'name': 'Denmark', 'id': '208', 'alpha3code':...",76,Christian,,Dannemann Eriksen,1624,1992-02-14,180,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Denmark', 'id': '208', 'alpha3code':...",54,right,C. Eriksen,7712


In [13]:
df_players.to_hdf('wyscout.h5', key = 'players', mode = 'a')

### Matches

In [14]:
competitions = [
    'England',
    #'France',
    #'Germany',
    #'Italy',
    'Spain',
    #'European Championship',
    #'World Cup'
]

In [15]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [16]:
df_matches.tail(10)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
370,Played,4406122,1,"{'701': {'scoreET': 0, 'coachId': 3485, 'side'...",181144,2017-08-21 20:00:00,701,Estadio La Rosaleda,2565553,"Málaga - Eibar, 0 - 1","August 21, 2017 at 10:00:00 PM GMT+2","[{'refereeId': 398913, 'role': 'referee'}, {'r...",Regular,795
371,Played,4406122,1,"{'695': {'scoreET': 0, 'coachId': 301696, 'sid...",181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,2565548,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795
372,Played,4406122,1,"{'675': {'scoreET': 0, 'coachId': 275283, 'sid...",181144,2017-08-20 20:15:00,675,Estadio Municipal de Riazor,2565555,"Deportivo La Coruña - Real Madrid, 0 - 3","August 20, 2017 at 10:15:00 PM GMT+2","[{'refereeId': 395062, 'role': 'referee'}, {'r...",Regular,795
373,Played,4406122,1,"{'684': {'scoreET': 0, 'coachId': 5298, 'side'...",181144,2017-08-20 18:15:00,676,Camp Nou,2565554,"Barcelona - Real Betis, 2 - 0","August 20, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 398919, 'role': 'referee'}, {'r...",Regular,795
374,Played,4406122,1,"{'698': {'scoreET': 0, 'coachId': 4107, 'side'...",181144,2017-08-20 16:15:00,0,San Mamés Barria,2565550,"Athletic Club - Getafe, 0 - 0","August 20, 2017 at 6:15:00 PM GMT+2","[{'refereeId': 420995, 'role': 'referee'}, {'r...",Regular,795
375,Played,4406122,1,"{'691': {'scoreET': 0, 'coachId': 169425, 'sid...",181144,2017-08-19 20:15:00,0,Estadio Ramón Sánchez Pizjuán,2565552,"Sevilla - Espanyol, 1 - 1","August 19, 2017 at 10:15:00 PM GMT+2","[{'refereeId': 378950, 'role': 'referee'}, {'r...",Regular,795
376,Played,4406122,1,"{'756': {'scoreET': 0, 'coachId': 167044, 'sid...",181144,2017-08-19 18:15:00,0,Estadi Municipal de Montilivi,2565551,"Girona - Atlético Madrid, 2 - 2","August 19, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 378994, 'role': 'referee'}, {'r...",Regular,795
377,Played,4406122,1,"{'692': {'scoreET': 0, 'coachId': 3880, 'side'...",181144,2017-08-19 16:15:00,687,Estadio de Balaídos,2565549,"Celta de Vigo - Real Sociedad, 2 - 3","August 19, 2017 at 6:15:00 PM GMT+2","[{'refereeId': 384946, 'role': 'referee'}, {'r...",Regular,795
378,Played,4406122,1,"{'674': {'scoreET': 0, 'coachId': 210074, 'sid...",181144,2017-08-18 20:15:00,674,Estadio de Mestalla,2565556,"Valencia - Las Palmas, 1 - 0","August 18, 2017 at 10:15:00 PM GMT+2","[{'refereeId': 381927, 'role': 'referee'}, {'r...",Regular,795
379,Played,4406122,1,"{'696': {'scoreET': 0, 'coachId': 117648, 'sid...",181144,2017-08-18 18:15:00,712,Estadio Municipal de Butarque,2565557,"Leganés - Deportivo Alavés, 1 - 0","August 18, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 398931, 'role': 'referee'}, {'r...",Regular,795


In [17]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

### Events

In [18]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index = False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key = f'events/match_{match_id}', mode = 'a')

## Convert event data into the SPADL representation

In [19]:
convert_to_spadl('wyscout.h5', 'spadl.h5')

...Inserting actiontypes
...Inserting bodyparts
...Inserting results
...Converting games
...Converting players


  0%|          | 1/760 [00:00<01:19,  9.60game/s]

...Converting teams
...Generating player_games


  0%|          | 0/760 [00:00<?, ?game/s]

...Converting events to actions


100%|██████████| 760/760 [10:50<00:00,  1.17game/s]


# Value Game states

In [20]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')

In [21]:
df_games.tail(10)

,game_id,competition_id,season_id,game_date,home_team_id,away_team_id
370,2565553,795,181144,2017-08-21 20:00:00,683,701
371,2565548,795,181144,2017-08-21 18:15:00,695,682
372,2565555,795,181144,2017-08-20 20:15:00,677,675
373,2565554,795,181144,2017-08-20 18:15:00,676,684
374,2565550,795,181144,2017-08-20 16:15:00,678,698
375,2565552,795,181144,2017-08-19 20:15:00,680,691
376,2565551,795,181144,2017-08-19 18:15:00,756,679
377,2565549,795,181144,2017-08-19 16:15:00,692,687
378,2565556,795,181144,2017-08-18 20:15:00,674,714
379,2565557,795,181144,2017-08-18 18:15:00,712,696


In [22]:
nb_prev_actions = 3

## Generate game state features

In [23]:
functions_features = [
    features.actiontype_onehot,
    features.bodypart_onehot,
    features.result_onehot,
    features.goalscore,
    features.startlocation,
    features.endlocation,
    features.movement,
    features.space_delta,
    features.startpolar,
    features.endpolar,
    features.team,
    features.time_delta
]

In [24]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    dfs_gamestates = features.gamestates(df_actions, nb_prev_actions=nb_prev_actions)
    dfs_gamestates = features.play_left_to_right(dfs_gamestates, game['home_team_id'])
    
    df_features = pd.concat([function(dfs_gamestates) for function in functions_features], axis=1)
    df_features.to_hdf('features.h5', key=f'game_{game_id}')

## Generate game state labels

In [25]:
functions_labels = [
    labels.scores,
    labels.concedes
]

In [26]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    df_labels = pd.concat([function(df_actions) for function in functions_labels], axis=1)
    df_labels.to_hdf('labels.h5', key=f'game_{game_id}')

## Generate dataset

In [27]:
columns_features = features.feature_column_names(functions_features, nb_prev_actions=nb_prev_actions)

In [28]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    dfs_features.append(df_features[columns_features])
df_features = pd.concat(dfs_features).reset_index(drop=True)

In [29]:
df_features.head(10)

,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,type_take_on_a0,type_foul_a0,type_tackle_a0,type_interception_a0,type_shot_a0,type_shot_penalty_a0,type_shot_freekick_a0,type_keeper_save_a0,type_keeper_claim_a0,type_keeper_punch_a0,type_keeper_pick_up_a0,type_clearance_a0,type_bad_touch_a0,type_non_action_a0,type_dribble_a0,type_goalkick_a0,type_pass_a1,type_cross_a1,type_throw_in_a1,type_freekick_crossed_a1,type_freekick_short_a1,type_corner_crossed_a1,type_corner_short_a1,type_take_on_a1,type_foul_a1,type_tackle_a1,type_interception_a1,type_shot_a1,type_shot_penalty_a1,type_shot_freekick_a1,type_keeper_save_a1,type_keeper_claim_a1,type_keeper_punch_a1,...,start_y_a1,start_x_a2,start_y_a2,end_x_a0,end_y_a0,end_x_a1,end_y_a1,end_x_a2,end_y_a2,dx_a0,dy_a0,movement_a0,dx_a1,dy_a1,movement_a1,dx_a2,dy_a2,movement_a2,dx_a01,dy_a01,mov_a01,dx_a02,dy_a02,mov_a02,start_dist_to_goal_a0,start_angle_to_goal_a0,start_dist_to_goal_a1,start_angle_to_goal_a1,start_dist_to_goal_a2,start_angle_to_goal_a2,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,34.00,52.50,34.00,42.00,37.40,42.00,37.40,42.00,37.40,-10.50,3.40,11.036757,-10.50,3.40,11.036757,-10.50,3.40,11.036757,-1.050000e+01,3.400000e+00,1.103676e+01,-10.50,3.400000e+00,11.036757,52.500000,0.000000,52.500000,0.000000,52.500000,0.000000,63.091679,0.053916,63.091679,0.053916,63.091679,0.053916,True,True,0.000000,0.000000
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,34.00,52.50,34.00,40.95,57.80,42.00,37.40,42.00,37.40,-1.05,20.40,20.427004,-10.50,3.40,11.036757,-10.50,3.40,11.036757,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000,63.091679,0.053916,52.500000,0.000000,52.500000,0.000000,68.328929,0.355773,63.091679,0.053916,63.091679,0.053916,True,True,1.997756,1.997756
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,37.40,52.50,34.00,32.55,47.60,40.95,57.80,42.00,37.40,-8.40,-10.20,13.213629,-1.05,20.40,20.427004,-10.50,3.40,11.036757,0.000000e+00,0.000000e+00,0.000000e+00,1.05,-2.040000e+01,20.427004,68.328929,0.355773,63.091679,0.053916,52.500000,0.000000,73.715416,0.185556,68.328929,0.355773,63.091679,0.053916,True,True,0.771744,2.769500
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,57.80,42.00,37.40,69.30,48.96,32.55,47.60,40.95,57.80,36.75,1.36,36.775156,-8.40,-10.20,13.213629,-1.05,20.40,20.427004,0.000000e+00,0.000000e+00,0.000000e+00,8.40,1.020000e+01,13.213629,73.715416,0.185556,68.328929,0.355773,63.091679,0.053916,38.707772,0.396818,73.715416,0.185556,68.328929,0.355773,True,True,2.174464,2.946208
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,47.60,40.95,57.80,74.55,55.76,69.30,48.96,32.55,47.60,5.25,6.80,8.590838,36.75,1.36,36.775156,-8.40,-10.20,13.213629,0.000000e+00,0.000000e+00,0.000000e+00,-36.75,-1.360000e+00,36.775156,38.707772,0.396818

In [30]:
columns_labels = [
    'scores',
    'concedes'
]

In [31]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    dfs_labels.append(df_labels[columns_labels])
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

In [32]:
df_labels.head(10)

,scores,concedes
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


##  Train Classifiers

In [33]:
%%time
models = {}
for column_labels in columns_labels:
    model = XGBClassifier(
        eval_metric='logloss',
        use_label_encoder=False,
    )
    model.fit(df_features, df_labels[column_labels])
    models[column_labels] = model

CPU times: user 11min 50s, sys: 4.68 s, total: 11min 55s
Wall time: 11min 51s


## Estimate probabilities

In [34]:
dfs_predictions = {}
for column_labels in columns_labels:
    model = models[column_labels]
    probabilities = model.predict_proba(df_features)
    predictions = probabilities[:, 1]
    dfs_predictions[column_labels] = pd.Series(predictions)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [35]:
df_predictions.head(10)

,scores,concedes
0,0.008324,0.002587
1,0.006611,0.002192
2,0.005190,0.002490
3,0.011728,0.002146
4,0.015702,0.001882
5,0.018653,0.001901
6,0.013932,0.004677
7,0.002852,0.031010
8,0.012309,0.003534
9,0.002142,0.013139


In [36]:
dfs_game_ids = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    dfs_game_ids.append(df_actions['game_id'])
df_game_ids = pd.concat(dfs_game_ids, axis=0).astype('int').reset_index(drop=True)

In [37]:
df_predictions = pd.concat([df_predictions, df_game_ids], axis=1)

In [38]:
df_predictions.head(10)

,scores,concedes,game_id
0,0.008324,0.002587,2500089
1,0.006611,0.002192,2500089
2,0.005190,0.002490,2500089
3,0.011728,0.002146,2500089
4,0.015702,0.001882,2500089
5,0.018653,0.001901,2500089
6,0.013932,0.004677,2500089
7,0.002852,0.031010,2500089
8,0.012309,0.003534,2500089
9,0.002142,0.013139,2500089


In [39]:
df_predictions_per_game = df_predictions.groupby('game_id')

In [40]:
for game_id, df_predictions in tqdm(df_predictions_per_game):
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions[columns_labels].to_hdf('predictions.h5', key=f'game_{game_id}')

# Value on-the-ball actions

In [41]:
df_players = pd.read_hdf('spadl.h5', key='players')
df_teams = pd.read_hdf('spadl.h5', key='teams')

In [42]:
dfs_values = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .merge(df_players, how='left')
        .merge(df_teams, how='left')
        .reset_index(drop=True)
    )
    
    df_predictions = pd.read_hdf('predictions.h5', key=f'game_{game_id}')
    df_values = value(df_actions, df_predictions['scores'], df_predictions['concedes'])
    
    df_all = pd.concat([df_actions, df_predictions, df_values], axis=1)
    dfs_values.append(df_all)

In [43]:
df_values = (pd.concat(dfs_values)
    .sort_values(['game_id', 'period_id', 'time_seconds'])
    .reset_index(drop=True)
)

In [44]:
df_values[
    ['short_name', 'scores', 'concedes', 'offensive_value', 'defensive_value', 'vaep_value']
].head(30)

,short_name,scores,concedes,offensive_value,defensive_value,vaep_value
0,A. Lacazette,0.005946,0.002813,0.000000,-0.000000,0.000000
1,R. Holding,0.008671,0.002230,0.002725,0.000583,0.003308
2,M. Özil,0.006589,0.002297,-0.002082,-0.000067,-0.002149
3,Mohamed Elneny,0.005146,0.002172,-0.001443,0.000125,-0.001318
4,Bellerín,0.011862,0.002003,0.006716,0.000169,0.006885
5,M. Özil,0.009294,0.003642,-0.002568,-0.001639,-0.004207
6,H. Maguire,0.005363,0.003057,0.001721,0.006236,0.007957
7,Bellerín,0.011354,0.002857,0.008296,0.002506,0.010803
8,Bellerín,0.010723,0.002267,-0.000630,0.000590,-0.000041
9,G. Xhaka,0.008913,0.002057,-0.001811,0.000210,-0.001600


# Rate Players

## Rate according to total VAEP value

In [45]:
df_ranking = (df_values[['player_id', 'team_name', 'short_name', 'vaep_value']]
    .groupby(['player_id', 'team_name', 'short_name'])
    .agg(vaep_count=('vaep_value', 'count'), vaep_sum=('vaep_value', 'sum'))
    .sort_values('vaep_sum', ascending=False)
    .reset_index()
)

In [46]:
df_ranking.head(10)

,player_id,team_name,short_name,vaep_count,vaep_sum
0,3359.0,FC Barcelona,L. Messi,2753,36.589249
1,120353.0,Liverpool FC,Mohamed Salah,1568,27.591764
2,8717.0,Tottenham Hotspur FC,H. Kane,1153,19.278978
3,3840.0,Real Club Celta de Vigo,Iago Aspas,1771,19.161940
4,3682.0,Club Atlético de Madrid,A. Griezmann,1394,18.488995
5,26150.0,Leicester City FC,R. Mahrez,2022,18.293406
6,38021.0,Manchester City FC,K. De Bruyne,3528,18.237783
7,3676.0,Real Sociedad de Fútbol,Illarramendi,3172,17.046106
8,25707.0,Chelsea FC,E. Hazard,1974,16.572348
9,54.0,Tottenham Hotspur FC,C. Eriksen,2959,16.475533


## Rate according to total VAEP value per 90 minutes

In [47]:
df_player_games = pd.read_hdf('spadl.h5', 'player_games')
df_player_games = df_player_games[df_player_games['game_id'].isin(df_games['game_id'])]

In [48]:
df_minutes_played = (df_player_games[['player_id', 'minutes_played']]
    .groupby('player_id')
    .sum()
    .reset_index()
)

In [49]:
df_minutes_played.head(10)

,player_id,minutes_played
0,33,92.903192
1,36,1238.400194
2,38,382.741398
3,48,3344.850202
4,54,3350.424005
5,56,267.266792
6,74,1515.130449
7,93,2941.903193
8,99,103.097068
9,107,2839.106630


In [50]:
df_ranking_p90 = df_ranking.merge(df_minutes_played)
df_ranking_p90 = df_ranking_p90[df_ranking_p90['minutes_played'] > 360]
df_ranking_p90['vaep_rating'] = df_ranking_p90['vaep_sum'] * 90 / df_ranking_p90['minutes_played']
df_ranking_p90 = df_ranking_p90.sort_values('vaep_rating', ascending=False)

In [51]:
df_ranking_p90.head(10)

,player_id,team_name,short_name,vaep_count,vaep_sum,minutes_played,vaep_rating
0,3359.0,FC Barcelona,L. Messi,2753,36.589249,3107.979597,1.059541
1,120353.0,Liverpool FC,Mohamed Salah,1568,27.591764,2996.529579,0.828712
12,8278.0,Real Madrid Club de Fútbol,G. Bale,1104,13.866236,1852.140087,0.673794
4,3682.0,Club Atlético de Madrid,A. Griezmann,1394,18.488995,2620.010232,0.635116
491,326523.0,Real Madrid Club de Fútbol,Dani Ceballos,446,2.941594,420.702252,0.629289
8,25707.0,Chelsea FC,E. Hazard,1974,16.572348,2505.438518,0.595309
16,8325.0,Manchester City FC,S. Agüero,982,13.461662,2038.785220,0.594251
38,280383.0,Levante UD,E. Bardhi,975,10.773466,1638.749361,0.591678
11,3322.0,Real Madrid Club de Fútbol,Cristiano Ronaldo,1182,15.224794,2357.056994,0.581331
3,3840.0,Real Club Celta de Vigo,Iago Aspas,1771,19.161940,3039.079676,0.567466


In [52]:
df_ranking_p90.to_csv('ranking.csv', index=False)